In [5]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [7]:
image = 'UAV_RGB_10cm.tif'


In [8]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs = {
    'points_per_side': 32,
    'pred_iou_thresh': 0.85,
    'stability_score_thresh': 0.92,
     'min_mask_region_area': 100,
},
)

In [10]:
mask = 'SAM_mask.tiff'

sam.generate(image, mask)


100%|██████████| 28/28 [03:18<00:00,  7.09s/it]


In [ ]:
shapefile = 'SAM_raw.shp'
sam.tiff_to_vector(mask, shapefile)

In [ ]:
#intersection_over_union - segment anything

import geopandas as gpd

def read_shapefile(file_path):
    return gpd.read_file(file_path)

def intersection_over_union(ground_truth, prediction):
    intersection = ground_truth.intersection(prediction)
    union = ground_truth.union(prediction)
    iou = intersection.area / union.area
    return iou

# Read the shapefiles
shapefile_a_path = "reference segmentation.shp"
shapefile_b_path = "SAM_raw.shp"
shapefile_a = read_shapefile(shapefile_a_path)
shapefile_b = read_shapefile(shapefile_b_path)



# Check if both shapefiles have the same CRS
if shapefile_a.crs != shapefile_b.crs:
    shapefile_b = shapefile_b.to_crs(shapefile_a.crs)

# Compute the IoU
iou = intersection_over_union(shapefile_a.unary_union, shapefile_b.unary_union)
print("Segment anything_IoU:", iou)

In [ ]:
#intersection_over_union - Clean shape

import geopandas as gpd

def read_shapefile(file_path):
    return gpd.read_file(file_path)

def intersection_over_union(ground_truth, prediction):
    intersection = ground_truth.intersection(prediction)
    union = ground_truth.union(prediction)
    iou = intersection.area / union.area
    return iou

# Read the shapefiles
shapefile_a_path = "reference segmentation.shp"
shapefile_b_path = "SAM_clean.shp"
shapefile_a = read_shapefile(shapefile_a_path)
shapefile_b = read_shapefile(shapefile_b_path)



# Check if both shapefiles have the same CRS
if shapefile_a.crs != shapefile_b.crs:
    shapefile_b = shapefile_b.to_crs(shapefile_a.crs)

# Compute the IoU
iou = intersection_over_union(shapefile_a.unary_union, shapefile_b.unary_union)
print("Clean shape IoU:", iou)


In [ ]:
vector = 'SAM_raw.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)


In [3]:
m = leafmap.Map(center=[44.193079, 5.157165], zoom=18)
m.add_basemap('SATELLITE')

vector = 'SAM_clean_segmentation.gpkg'
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.2,
}
m.add_vector(vector, layer_name='Vector', style=style)
m

Map(center=[44.193079, 5.157165], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …